In [ ]:
# Imports
!pip install yfinance
!pip install numba
import yfinance as yf
import numpy as np
from collections import deque

     |████████████████████████████████| 6.3 MB 49.7 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.64-py2.py3-none-any.whl size=24109 sha256=a3bb7f705e003b1aa622dff5e410cc7147e5e58040a4af3be07736ce88af1e8c
  Stored in directory: /root/.cache/pip/wheels/86/fe/9b/a4d3d78796b699e37065e5b6c27b75cff448ddb8b24943c288
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
snp = yf.Ticker("^GSPC") # Initialize Ticker

snp_history = snp.history(period="max", interval="1wk") # Download history of ticker

                   Open         High  ...  Dividends  Stock Splits
Date                                  ...                         
1950-01-02    16.660000    17.090000  ...          0             0
1950-01-09    17.080000    17.090000  ...          0             0
1950-01-16    16.719999    16.940001  ...          0             0
1950-01-23    16.920000    16.920000  ...          0             0
1950-01-30    17.020000    17.350000  ...          0             0
...                 ...          ...  ...        ...           ...
2021-10-11  4385.439941  4475.819824  ...          0             0
2021-10-18  4463.720215  4559.669922  ...          0             0
2021-10-25  4553.689941  4608.080078  ...          0             0
2021-11-01  4610.620117  4620.339844  ...          0             0
2021-11-02  4613.339844  4633.060059  ...          0             0

[3750 rows x 7 columns]


In [ ]:
# Data narrowed into Index, Date, Close, Volume
snp_data = snp_history[["Close", "Volume"]]
snp_data = snp_data.reset_index()

In [7]:
# Add new W_Average Column to the data (Weighted 30 week moving average)
def calculate_waverage(close):
  #Declaration of variables:
  res = np.zeros(close.shape)
  weights = range(1,31)
  total_weight = sum(weights)

  past30 = deque()  # Initialize queue in which to keep past 30 values
  
  # Add first 29 values
  for i in range(0,29): 
    past30.append(close[i])

  # Calculate weighted average. Updating and dequeing 1 value with every loop.
  for i in range(29, close.shape[0]):  
    past30.append(close[i])
    
    res[i] = sum([value * weight for value, weight in zip(past30,weights)]) / total_weight


    past30.popleft()
  
  return res

snp_data['W_Average'] = calculate_waverage(snp_data['Close'])
snp_data.to_csv(path_or_buf="snp_data_1wk_weighted30wk")  # Export data to a csv which will then be used for rest of project.